In [1]:
# Zum Speichern der Plots des kurzen Bleches aus Rohdaten, zentrierten Daten und stationären Bereich
from PreProcessing_SingleBlech_Function_KurzesBlech import Preprocessing

for i in range (1,8):
    Data_stat, Data_zentriert, BlechNummer = Preprocessing(i,0,1400,1)

C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\15_Plots\Plots_KurzesBlech_1 --> Existiert bereits
C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\15_Plots\Plots_KurzesBlech_2 --> Existiert bereits
C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\15_Plots\Plots_KurzesBlech_3 --> Existiert bereits
C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\15_Plots\Plots_KurzesBlech_4 --> Existiert bereits
C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\15_Plots\Plots_KurzesBlech_5 --> Existiert bereits
C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\15_Plots\Plots_KurzesBlech_6 --> Existiert bereits
C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\15_Plots\Plots_KurzesBlech_7 --> Existiert bereits


In [ ]:
# Reinladen der Daten für das beste CNN Modell im nächsten Code Abschnitt
from Load_Data_for_Modelling import Get_data

# Hier werden die interpolierten Daten reingeladen
df_Int, Interpoliertes_df = Get_data(0,1800,1,2)

# Skalierung und Fensterung der Daten
from Fensterung_Scaling_DeepLearning import Fensterung_Scale
X_train_scaled, X_val_scaled, X_test_scaled, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels, X_test_scaled_int, Y_test_interpolation, Blechnummern_Test_Int = Fensterung_Scale(df_Int, interpoliertesdf=Interpoliertes_df, Validation_data=1, random=1, Train_Test_Split=2, window_size=10, Interpolation=1, size=0.1)

In [5]:
# Modellbildung bestes CNN, hier kann das Modell mit den vorher definierten Zufallsvariablen und Größen gespeichert werden
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras import layers
from Load_Data_for_Modelling_Function import Data_for_Model
from Splitting_Scaling_Function import Split_Scaling
import matplotlib.pyplot as plt
import kerastuner as kt
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import shutil
import os
import json
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import gc

Y_train = np.squeeze(Y_train)
Y_test = np.squeeze(Y_test)
Y_val =np.squeeze(Y_val)
Y_train_scaled = np.squeeze(Y_train_scaled)
Y_val_scaled = np.squeeze(Y_val_scaled)

# TensorFlow-Sitzung zurücksetzen
tf.keras.backend.clear_session()
gc.collect()

# CNN Modell definieren
input_layer = layers.Input(shape=(10,11))


#Hyperparametersuche 2 60 Trials
conv_1 = layers.Conv1D(filters=160, kernel_size=2, activation='relu', padding='same', kernel_regularizer=keras.regularizers.l2(0.01))(input_layer)
pool_1 = layers.MaxPooling1D(pool_size=2)(conv_1)
conv_2 = layers.Conv1D(filters=480, kernel_size=4, activation='relu', padding='same', strides=1, kernel_regularizer=keras.regularizers.l2(0.01))(pool_1)
pool_2 = layers.MaxPooling1D(pool_size=2)(conv_2)
    
    
flatten = layers.Flatten()(pool_2)

#Hyperparametersuche 3
dense_layer = layers.Dense(64, activation='tanh', kernel_regularizer=keras.regularizers.l2(0.01))(flatten)

# Output Layers definieren
X_output = layers.Dense(1, activation='linear', name='Verstellweg_X')(dense_layer)
Y_output = layers.Dense(1, activation='linear', name='Verstellweg_Y')(dense_layer)
Phi_output = layers.Dense(1, activation='linear', name='Verstellweg_Phi')(dense_layer)

# Liste erstellen für alle Outputs
outputs = [X_output, Y_output, Phi_output]

# Modell definieren 
model = keras.Model(inputs=input_layer, outputs=outputs)

# Hyperparameter 3 60 Trials
model.compile(optimizer=keras.optimizers.Adam(0.0003255639325303961), 
            loss=['mean_absolute_error', 'mean_absolute_error', 'mean_absolute_error'], 
            metrics={'Verstellweg_X': 'mae', 'Verstellweg_Y': 'mae', 'Verstellweg_Phi': 'mae'})

# Modell zusammenfassen
#model.summary()

# Early Stopping Callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

CNN = model.fit(X_train_scaled, [Y_train[:,0], Y_train[:,1], Y_train[:,2]],
                        epochs=30,
                        validation_data=(X_val_scaled, [Y_val[:, 0], Y_val[:, 1], Y_val[:, 2]]),
                        callbacks=[early_stopping])

# Speichern des Modells
Pfad = r'C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\14_Modelle_Hyperparameter'
model_pfad = os.path.join(Pfad, 'Best_Modell_CNN_Interpolationsfaktor2_Random1.h5')
model.save(model_pfad)

# Vorhersagen für den Testdatensatz erstellen
predictions1 = model.predict(X_test_scaled)
X_p, Y_p, Phi_p = predictions1

# predictions_int = model.predict(X_test_scaled_int)
# X_int, Y_int, Phi_int = predictions_int

 # MAE für den Testdatensatz berechnen
mae_X = mean_absolute_error(Y_test[:,0], X_p)
mae_Y = mean_absolute_error(Y_test[:,1], Y_p)
mae_Phi = mean_absolute_error(Y_test[:,2], Phi_p)

print(f"Mean Absolute Error for Verstellweg_X_ alleDaten: {mae_X}")
print(f"Mean Absolute Error for Verstellweg_Y_alleDaten: {mae_Y}")
print(f"Mean Absolute Error for Verstellweg_Phi_alleDaten: {mae_Phi}")

Epoch 1/30
19746/19746 ━━━━━━━━━━━━━━━━━━━━ 313s 15ms/step - Verstellweg_Phi_mae: 0.3729 - Verstellweg_X_mae: 0.0240 - Verstellweg_Y_mae: 0.3259 - loss: 1.3836 - val_Verstellweg_Phi_mae: 0.2428 - val_Verstellweg_X_mae: 0.0084 - val_Verstellweg_Y_mae: 0.2906 - val_loss: 0.7375
Epoch 2/30
19746/19746 ━━━━━━━━━━━━━━━━━━━━ 322s 16ms/step - Verstellweg_Phi_mae: 0.1912 - Verstellweg_X_mae: 0.0049 - Verstellweg_Y_mae: 0.1690 - loss: 0.5505 - val_Verstellweg_Phi_mae: 0.2290 - val_Verstellweg_X_mae: 0.0087 - val_Verstellweg_Y_mae: 0.2900 - val_loss: 0.6922
Epoch 3/30
19746/19746 ━━━━━━━━━━━━━━━━━━━━ 325s 16ms/step - Verstellweg_Phi_mae: 0.1738 - Verstellweg_X_mae: 0.0045 - Verstellweg_Y_mae: 0.1549 - loss: 0.4933 - val_Verstellweg_Phi_mae: 0.2530 - val_Verstellweg_X_mae: 0.0159 - val_Verstellweg_Y_mae: 0.2975 - val_loss: 0.7165
Epoch 4/30
19746/19746 ━━━━━━━━━━━━━━━━━━━━ 305s 15ms/step - Verstellweg_Phi_mae: 0.1633 - Verstellweg_X_mae: 0.0042 - Verstellweg_Y_mae: 0.1485 - loss: 0.4630 - val_Ver

1119/1119 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step
Mean Absolute Error for Verstellweg_X_ alleDaten: 0.009507456149995222
Mean Absolute Error for Verstellweg_Y_alleDaten: 0.26674271942657424
Mean Absolute Error for Verstellweg_Phi_alleDaten: 0.13997678431677085


In [2]:
from Load_Data_for_Modelling_Function_KurzesBlech import Data_for_Model
from Fensterung_Scaling_DeepLearning import Fensterung_Scale
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error
import os
import numpy as np
import random
import pandas as pd

# Bestes CNN Modell (siehe Code Abschnitt zuvor) laden
Pfad = r'C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\14_Modelle_Hyperparameter\CNN'
model_pfad = os.path.join(Pfad, 'Best_Modell_CNN_Interpolationsfaktor2_Random1.h5')
model = load_model(model_pfad)

# Reinladen der Daten des kurzen Bleches
data = Data_for_Model(0,1400)
print(data)
# Generieren einer Liste von 10 eindeutigen zufälligen Ganzzahlen zwischen 0 und 100
random.seed(2)
Random_numbers = random.sample(range(101), 10)
print(Random_numbers)

# Leere Liste für die Fehler
Liste_MAEs_Blechsplit = []
Liste_Fehler_Blechsplit = []

# 10 fache Validierung für das kurze Blech (Erfolgt hierbei egal nach welchem Split)
for i in Random_numbers:
    # Skaliere und Fenster die Daten entsprechend des gewünschten Zufallsfaktors
    X_train_scaled, X_test_scaled, Y_train_scaled, Y_test_scaled, Y_train, Y_test, scalers_features, scaler_labels, Blechnummern_Test = Fensterung_Scale(data, Validation_data=0, random=i, Train_Test_Split=1, window_size=10, size=0.01, Datengröße=1400)

    print(X_test_scaled.shape)
    print(X_test_scaled.shape)

    # Squeeze damit kein Fehler beim Vergleich der Daten auftritt
    Y_train = np.squeeze(Y_train)
    Y_test = np.squeeze(Y_test)

    # Vorhersagen für den Testdatensatz erstellen
    # Hier wird mit Train die Daten predicted weil die Size sehr klein gewählt wird beim SPlitten sodass möglichst alle Daten genutzt werden und die Testdaten eben sehr klein sind 
    # Somit werden über 99% der Daten für den Test des kurzen Bleches hier verwendet
    predictions1 = model.predict(X_train_scaled)
    X_p, Y_p, Phi_p = predictions1

    # predictions_int = model.predict(X_test_scaled_int)
    # X_int, Y_int, Phi_int = predictions_int

    # MAE für den Testdatensatz berechnen
    mae_X = mean_absolute_error(Y_train[:,0], X_p)
    mae_Y = mean_absolute_error(Y_train[:,1], Y_p)
    mae_Phi = mean_absolute_error(Y_train[:,2], Phi_p)

    # print(f"Mean Absolute Error for Verstellweg_X_ alleDaten: {mae_X}")
    # print(f"Mean Absolute Error for Verstellweg_Y_alleDaten: {mae_Y}")
    # print(f"Mean Absolute Error for Verstellweg_Phi_alleDaten: {mae_Phi}")
    
    # Hinzufügen der MAEs zum Dataframe
    MAE_BlechSplit_df = pd.DataFrame([{'CV':i, 'Error' : 'MAE', 'X': mae_X, 'Y': mae_Y, 'Phi': mae_Phi}])
    Liste_MAEs_Blechsplit.append(MAE_BlechSplit_df)
    
    # Berechnen der einzelnen Fehler aus Label und Prädiktion
    Fehler_X = Y_train[:,0]-X_p[:,0]
    Fehler_Y = Y_train[:,1]-Y_p[:,0]
    Fehler_Phi = Y_train[:,2]-Phi_p[:,0]
    
    # Fehler in einen DataFrame konvertieren
    df_Fehler = pd.DataFrame({
        'Label_X': Y_train[:,0],
        'Label_Y': Y_train[:,1],
        'Label_Phi': Y_train[:,2],
        'Fehler_X': Fehler_X,
        'Fehler_Y': Fehler_Y,
        'Fehler_Phi': Fehler_Phi})
    
    # Appende die Splitmethode und den Durchlauf zur Fehlerliste
    df_Fehler.insert(loc=0, column='SplitMethode', value='Blechsplit')
    df_Fehler.insert(loc=1, column='CV', value=i)
    Liste_Fehler_Blechsplit.append(df_Fehler)

# Füge die Listen zusammen
MAE_Blechsplit = pd.concat(Liste_MAEs_Blechsplit, ignore_index=True)
Fehler_Blechsplit_df = pd.concat(Liste_Fehler_Blechsplit, ignore_index=True)

Anzahl der Zeilen im finalen DataFrame: 4200
None
      1-OW-OS Lateral Force  1-OW-OS Axial Force  2-OW-RS Lateral Force  \
0               2311.959854            16.310887            6104.752182   
1               2250.925952             0.542466            6030.260740   
2               2314.176390             2.909735            6033.506382   
3               2299.452258            11.415856            6035.168784   
4               2307.051810            22.690478            6033.427220   
...                     ...                  ...                    ...   
4195            1889.701038            23.787647            5314.785876   
4196            1853.919814             7.417377            5357.850004   
4197            1855.898864             9.824770            5291.749734   
4198            1852.019926            23.346292            5243.856724   
4199            1867.931488            31.611673            5332.043192   

      2-OW-RS Axial Force  3-UW-RS Lateral Force 

In [3]:
# Hier können die zuvor erstellten Listen gespeichert werden in einer Excel

# Sicherungsordner hier angeben
Ordner = r'C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\13_ExcelvonDaten_Code\Evaluation_kurzesBlech'

# print(Fehler_Blechsplit_df)
# print(MAE_Blechsplit)
# Berechnung der Standardabweichung und des Mittelwertes
Mean = MAE_Blechsplit[['X','Y','Phi']].mean() 
Std = MAE_Blechsplit[['X','Y','Phi']].std()

# Zusammenfügen der Mittelwerte und Std
MAE_BlechSplit = pd.concat([MAE_Blechsplit, pd.DataFrame([{'CV': 'Mittelwert', 'Error' : 'MAE', 'X': Mean[0], 'Y': Mean[1], 'Phi': Mean[2]}])], ignore_index=True)
MAE_BlechSplit_comp = pd.concat([MAE_BlechSplit, pd.DataFrame([{'CV': 'Standardabweichung', 'Error' : 'MAE', 'X': Std[0], 'Y': Std[1], 'Phi': Std[2]}])], ignore_index=True)

# print(MAE_BlechSplit)
# print(MAE_StandardSplit)

# Umwandeln von Flaot in String für die richtige Darstellung in der CSV
for Column in MAE_BlechSplit_comp.columns:
         MAE_BlechSplit_comp[Column] = MAE_BlechSplit_comp[Column].astype(str).str.replace('.', ',')
        
for Column in Fehler_Blechsplit_df:
        Fehler_Blechsplit_df[Column] = Fehler_Blechsplit_df[Column].astype(str).str.replace('.', ',')

# Speichern der Fehlerliste und der MAEs im entsprechenden Ordner
Fehler_Blechsplit_df.to_csv(f'{Ordner}\Fehler_CNNmitIntfaktor2_neuesBlech.csv', index=True, sep=';')

MAE_BlechSplit_comp.to_csv(f'{Ordner}\MAE_CNNmitIntfaktor2_neuesBlech.csv', index=True, sep=';')

C:\Users\corvi\AppData\Local\Temp\ipykernel_27164\1360701161.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  MAE_BlechSplit_comp[Column] = MAE_BlechSplit_comp[Column].astype(str).str.replace('.', ',')
C:\Users\corvi\AppData\Local\Temp\ipykernel_27164\1360701161.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Fehler_Blechsplit_df[Column] = Fehler_Blechsplit_df[Column].astype(str).str.replace('.', ',')
